# Applied Data Science Capstone Project


In [1]:
import pandas as pd
import numpy as np
import urllib.request 
from bs4 import BeautifulSoup
#!pip install geocoder
import geocoder # 

## Get Data From Wikipedia

In [2]:
f=urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html=f.read()



### Parse with Beautiful Soup

Get table rows, skipping column name row

In [3]:
soup=BeautifulSoup(html,'lxml')
table=soup.find_all("table")[0]
headerRow=table.tr
data=headerRow.find_next_siblings("tr")

### Create Dataframe

In [4]:
# Put Data in lists
PostalCode=list()
Burough=list()
Neighborhood=list()

#populate lists
for row in data:
    PostalCode.append(row("td")[0].get_text().strip())
    Burough.append(row("td")[1].get_text().strip())
    Neighborhood.append(row("td")[2].get_text().strip())
    
#Create DataFrame with lists
df=pd.DataFrame(np.column_stack([PostalCode,Burough,Neighborhood]),
             columns=['PostalCode','Burough','Neighborhood'])

### Remove rows without Burough assigned.

Where Neighborhood is unassigned, make equal to Burough

In [5]:

df=df[df['Burough']!='Not assigned']

for index,row in df.iterrows():
    if (row['Neighborhood']=="Not assigned"):
        row['Neighborhood']=row['Burough']

### Combine Neighborhoods in same PostalCode 

In [6]:
#Combine Neighborhoods in same PostalCode        
df2=pd.DataFrame(columns=['PostalCode','Burough','Neighborhood'])        
        
for index,row in df.iterrows():
    df2.loc[len(df2)]=[row['PostalCode'],row['Burough'],'']
    
df2.drop_duplicates('PostalCode',inplace=True)
df2.reset_index(drop=True,inplace=True)

for index,row in df2.iterrows():
    temp=df[df['PostalCode']==row['PostalCode']]
    for indexTemp,rowTemp in temp.iterrows():
        row['Neighborhood']+=", "+rowTemp['Neighborhood']
    row['Neighborhood']=row['Neighborhood'].lstrip(',')
        
df2.head()

,PostalCode,Burough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df2.shape

(103, 3)

# put Geospatial_Coordinates.csv file into dataframe

In [38]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_XXX = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XXX',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_XXX.get_object(Bucket='dsibmcapstone-donotdelete-pr-bq4niuqdmysuzq',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

In [35]:
zip_df=df_data_1.copy(deep=True)


## Add latitude and longitude data to neighborhood dataframe

In [37]:
dfn=df2.copy(deep=True)

dfn['Latitude']=None
dfn['Longitude']=None

for index,row in dfn.iterrows():
    
    row['Latitude']=float(zip_df[df_data_1['Postal Code']==row['PostalCode']]['Latitude'])
    row['Longitude']=float(zip_df[df_data_1['Postal Code']==row['PostalCode']]['Longitude'])
    
dfn.head()

,PostalCode,Burough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
